# Initial Processing

In [1]:
%run ./1_Definitions.ipynb

thunder  Fri Jan 17 14:29:16 2020
[0] GeForce GTX 1080 Ti | 39'C,   0 % | 10767 / 11178 MB | hemelinr(10757M)
[1] GeForce GTX 1080 Ti | 24'C,   0 % | 10767 / 11178 MB | hemelinr(10757M)
[2] GeForce GTX 1080 Ti | 26'C,   0 % |    10 / 11178 MB |
[3] GeForce GTX 1080 Ti | 24'C,   0 % | 10936 / 11178 MB | hemelinr(10926M)
[4] GeForce GTX 1080 Ti | 23'C,   0 % | 10767 / 11178 MB | hemelinr(10757M)
[5] GeForce GTX 1080 Ti | 26'C,   0 % |    10 / 11178 MB |
[6] GeForce GTX 1080 Ti | 59'C,  21 % | 10895 / 11178 MB | michiel(10885M)
[7] GeForce GTX 1080 Ti | 43'C,   0 % | 10895 / 11178 MB | michiel(10885M)


/ds/environments/python2020/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.
/ds/environments/python2020/lib/python3.7/site-packages/keras/callbacks/callbacks.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


Dataset consist of 85730 patients data with 6251 features for each patient
Function definitions executed


In [2]:
!nvidia-smi
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "7"   # choose here your GPU (0-7)

import tensorflow as tf
import tensorflow.compat.v1 as tf
import keras
# from keras import backend as K
import tensorflow.python.keras.backend as K

Fri Jan 17 14:29:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 29%   39C    P2    56W / 250W |  10767MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 29%   24C    P8     7W / 250W |  10767MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
df = fundus_df.copy()

In [4]:
# Indices -  used to separate the data into train - validation and test sets
indices = np.arange(n_patients)

# 1. META DATA

## Handling 'None', 'NaN' and other values - DONE
Below code is commented out since the preprocessing is done and saved after label encoding - time taking process

In [5]:
# df.replace(to_replace=[None], value=np.nan, inplace=True) # Replace all None values by NaN

# # Replace Nan values depending on dtype of each column
# float_cols = df.select_dtypes(include=['float64']).columns
# int_cols = df.select_dtypes(include=['int64']).columns
# str_cols = df.select_dtypes(include=['object']).columns

# df.loc[:, int_cols] = df.loc[:, int_cols].fillna(0)
# df.loc[:, float_cols] = df.loc[:, float_cols].fillna(0)
# df.loc[:, str_cols] = df.loc[:, str_cols].fillna('')

# # Handling weird values
# df.iloc[:,17] = df.iloc[:,17].replace(regex=['^\.\.'], value='00') # changing '..2870' to '002870' by looking at the pattern

## Label Encoding - DONE
Saved file enc_df can be used directly to save time

In [6]:
# # Label Encoding for the non numeric columns
# enc_df = df.copy()
# for i in range(enc_df.shape[1]):
#     if not is_numeric_dtype(enc_df.iloc[:,i]):
#         enc_df.iloc[:,i] = le.fit_transform(enc_df.iloc[:,i])

# SAVING
# enc_df.to_pickle(DATA_dir + 'enc_df.pkl')


## Loading the encoded data

In [7]:
#LOAD
enc_df = pd.read_pickle(DATA_dir + 'enc_df.pkl')

# 2. Ground Truth Data Loading

In [21]:
# Since we are using UNION as our Ground truth -  we go with y_union
y = np.load(DATA_dir + 'GroundTruths/GT_intersect.npy')
y_union = np.load(DATA_dir + 'GroundTruths/GT_union.npy')

GTu_L = np.load(DATA_dir + 'GroundTruths/GTu_L.npy')               # Ground truth of left eye - union
GTu_R = np.load(DATA_dir + 'GroundTruths/GTu_R.npy')               # Ground truth of right eye - union

GTi_L = np.load(DATA_dir + 'GroundTruths/GTi_L.npy')
GTi_R = np.load(DATA_dir + 'GroundTruths/GTi_R.npy')

GT523_L = np.load(DATA_dir + 'GroundTruths/GT523_L.npy')
GT523_R = np.load(DATA_dir + 'GroundTruths/GT523_R.npy')

GT_19 = np.load(DATA_dir + 'GroundTruths/GT_19.npy')

In [22]:
GT523_L_df = pd.DataFrame(GT523_L, columns=['GT523_L'])
GT19_L_df = pd.DataFrame(GT_19, columns=['GT19_L'])
GTi_L_df = pd.DataFrame(GTi_L, columns=['GTi_L'])
GTu_L_df = pd.DataFrame(GTu_L, columns=['GTu_L'])

GT523_R_df = pd.DataFrame(GT523_R, columns=['GT523_R'])
GT19_R_df = pd.DataFrame(GT_19, columns=['GT19_R'])
GTi_R_df = pd.DataFrame(GTi_R, columns=['GTi_R'])
GTu_R_df = pd.DataFrame(GTu_R, columns=['GTu_R'])


In [23]:
GT_LEFT = pd.concat([GT523_L_df,GT19_L_df,GTi_L_df,GTu_L_df],axis = 1)
GT_RIGHT = pd.concat([GT523_R_df,GT19_R_df,GTi_R_df,GTu_R_df],axis = 1)

In [24]:
n_NG = 0
n_G = 0

for i in range(len(y_union)):
    if(y_union[i] == 1):
        n_G += 1
    else:
        n_NG += 1
        
print("UNION") # y_union is used for union
print("No. of Glaucoma patients in training set:",n_G)
print("No. of non-Glaucoma patients in training set:",n_NG)


UNION
No. of Glaucoma patients in training set: 4182
No. of non-Glaucoma patients in training set: 81548


# 3. FUNDUS Images filename Loading

In [25]:
# LOAD
filenames_L = np.load(DATA_dir + 'filenames_L.npy') # fundus left eye image names
filenames_R = np.load(DATA_dir + 'filenames_R.npy') # fundus right eye image names

# converted to dataframe for easier usage
filenames_L_df = pd.DataFrame(filenames_L, columns=['filenames_L']) 
filenames_R_df = pd.DataFrame(filenames_R, columns=['filenames_R'])

# 4. Concatenate File names and Ground truths
This is done so that file and respective ground truths can be accessed easily.

In [26]:
fundus_names_gt_df = pd.concat([filenames_L_df,GT_LEFT,filenames_R_df,GT_RIGHT], axis = 1)

In [27]:
fundus_names_gt_df.head()

,filenames_L,GT523_L,GT19_L,GTi_L,GTu_L,filenames_R,GT523_R,GT19_R,GTi_R,GTu_R
0,1000132_21015_0_0.png,1,0,0,0,1000132_21016_0_0.png,1,0,0,0
1,1000277_21015_0_1.png,1,0,0,0,1000277_21016_0_1.png,1,0,0,0
2,1000472_21015_0_0.png,1,0,0,0,1000472_21016_0_0.png,1,0,0,0
3,1000487_21015_0_0.png,1,0,0,0,1000487_21016_0_0.png,1,0,0,0
4,1000684_21015_0_0.png,1,0,0,0,1000684_21016_0_0.png,1,0,0,0


# 4. Splitting into Train Test Validation

Train-validation-test sets are created in such a way that a paritcular set of patients in each set, 
have the respective meta-data as well as left and right fundus image names along with their ground truth in that set.
This is done using indices and fundus_names_gt_df dataframe created in above field.

In [28]:
# Spliting to train and test
indices_train, indices_test, fundus_train, fundus_test = train_test_split(indices,fundus_names_gt_df,test_size=0.2, random_state=1)

# Spliting train into train and validation
indices_train, indices_val, fundus_train, fundus_val = train_test_split(indices_train,fundus_train,test_size=0.25, random_state=1)

print("Meta data index:",indices_train.shape, indices_val.shape, indices_test.shape)
print("Fundus Images and all ground truths:",fundus_train.shape, fundus_val.shape, fundus_test.shape)

Meta data index: (51438,) (17146,) (17146,)
Fundus Images and all ground truths: (51438, 10) (17146, 10) (17146, 10)


### GT union Split into Train Test Val as per indices

In [30]:
# TRAIN
yu_train = pd.DataFrame(columns = ['ClassLabel'])
for idx in range(len(indices_train)):
    yu_train.loc[idx] = y_union[indices_train[idx]]
    update_progress(idx/len(indices_train))

# VALIDATION
yu_val = pd.DataFrame(columns = ['ClassLabel'])
for idx in range(len(indices_val)):
    yu_val.loc[idx] = y_union[indices_val[idx]]
    update_progress(idx/len(indices_val))

# TEST
yu_test = pd.DataFrame(columns = ['ClassLabel'])
for idx in range(len(indices_test)):
    yu_test.loc[idx] = y_union[indices_test[idx]]
    update_progress(idx/len(indices_test))


Progress: [####################] 100.0%


In [31]:
print("Y_union:",yu_train.shape, yu_val.shape, yu_test.shape)
print("Fundus image names and GT:",fundus_train.shape, fundus_val.shape, fundus_test.shape)

Y_union: (51438, 1) (17146, 1) (17146, 1)
Fundus image names and GT: (51438, 10) (17146, 10) (17146, 10)


### Saving the train test validation data - DONE !!

In [32]:
# INDICES
np.save(tts_path + 'indices_train.npy', indices_train)
np.save(tts_path + 'indices_val.npy', indices_val)
np.save(tts_path + 'indices_test.npy', indices_test)

# FUNDUS IMAGE NAMES AND GROUND TRUTHS - LEFT and RIGHT
fundus_train.to_pickle(tts_path + 'fundus_train.pkl')
fundus_val.to_pickle(tts_path + 'fundus_val.pkl')
fundus_test.to_pickle(tts_path + 'fundus_test.pkl')

# GROUND TRUTH UNION FOR FEATURE IMPORTANCE
yu_train.to_pickle(tts_path + 'yu_train.pkl')
yu_val.to_pickle(tts_path + 'yu_val.pkl')
yu_test.to_pickle(tts_path + 'yu_test.pkl')